# ASMSA: Prepare and check input files

**Next steps**
- [tune.ipynb](tune.ipynb): Perform initial hyperparameter tuning for this molecule
- [train.ipynb](train.ipynb): Use results of previous tuning in more thorough training
- [md.ipynb](md.ipynb): Use a trained model in MD simulation with Gromacs

In [1]:
import mdtraj as md
import numpy as np
import urllib.request
import asmsa

2023-02-16 15:07:41.404366: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-16 15:07:41.496392: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Prepare input files

Tryptophan cage files are downloaded in this section from our Google drive. 

This is for demonstration purpose, in real use the inputs should be placed here, and _conf, traj, topol, index_ variables set to their filenames names.

In [2]:
# download trpcage files
urllib.request.urlretrieve("https://drive.google.com/uc?export=download&id=19RZmGgz9goXEAbreUd0OBCKWr5UAVN5d","index_correct.ndx")
urllib.request.urlretrieve("https://drive.google.com/uc?export=download&id=1G-4HRnc1R-LqAArFh0DTY-e5ULd8Goly","topol_correct.top")
urllib.request.urlretrieve("https://drive.google.com/uc?export=download&id=1ddOJPQxXCw3jY3Yds6bm-EwGps8ClVGQ","trpcage_correct.pdb")
urllib.request.urlretrieve("https://drive.google.com/uc?export=download&id=1FRM3-bCdbesShVcyRfk0cj0ILBL1ycbb","trpcage_red.xtc")
urllib.request.urlretrieve("https://drive.google.com/uc?export=download&id=1A28ik84vDhIzyHPHrL8nTxgjzOQAmzLU","trpcage_correct.gro")

('trpcage_correct.gro', <http.client.HTTPMessage at 0x7f1a08b56a70>)

In [ ]:
# Define input files

# input conformation
#conf = "alaninedipeptide_H.pdb"
conf = "trpcage_correct.pdb"

# input trajectory
# atom numbering must be consistent with {conf}

#traj = "alaninedipeptide_reduced.xtc"
traj = "trpcage_red.xtc"

# input topology
# expected to be produced with 
#    gmx pdb2gmx -f {conf} -p {topol} -n {index} -o {gro}

# Gromacs changes atom numbering, the index file must be generated and used as well
# gro file is used to generate inverse indexing for plumed.dat

#topol = "topol.top"
topol = "topol_correct.top"
index = 'index_correct.ndx'
gro = 'trpcage_correct.gro'

## Sanity checks

In [ ]:
# Load the trajectory, it should report expected numbers of frames and atoms/residua

tr = md.load(traj,top=conf)
idx=tr[0].top.select("name CA")

# for trivial cases like Ala-Ala, where superposing on CAs fails
#idx=tr[0].top.select("element != H")

tr.superpose(tr[0],atom_indices=idx)

In [ ]:
# Visual check, all frames should look "reasonable"

# Because of different conventions of numbering atoms in proteins,
# PDB file {conf} and the trajectory {traj} can become inconsistent, and this would appear here 
# as rather weird shapes of the molecule

import nglview as nv

v = nv.show_mdtraj(tr)
v.clear()
v.add_representation("licorice")
v

## Internal coordinates computation

Exercise the ASMSA library on your input. Just check everything appears to work.

There are multiple options that can be combined:
- use traditional internal coordinates (bond distances, angles, and dihedrals) or not
- include additional distances between atoms that may not be bound to express protein folding state more directly
   - dense (all-to-all) atom distances, feasible for very small peptides only
   - sparse atom distances (only some pairs are chosen)
   
**Choose the suitable one in the cell bellow, and copy the same to [tune.ipynb](tune.ipynb) and [train.ipynb](train.ipynb)**, they must be consistent


In [ ]:
# reshuffle the geometry to get frame last so that we can use vectorized calculations

geom = np.moveaxis(tr.xyz ,0,-1)
geom.shape

In [ ]:

# traditional internal coordinates (bond distances, angles, and torsions) only

# mol = asmsa.Molecule(conf,topol)

# internal coordinates and sparse any-any atom distances (not restricted to bonds)
# eventually, top (and index) can be left out to use sparse distances only

# in how many distances each atom should be involved in average
density = 2 # integer in [1, n_atoms-1]

sparse_dists = asmsa.NBDistancesSparse(geom.shape[0], density=density)
mol = asmsa.Molecule(pdb=conf,top=topol,ndx=index,fms=[sparse_dists])

# dense distances are feasible for very small (upto 5 residua) peptides only

# dense_dists = asmsa.NBDistancesDense(geom.shape[0])
# mol = asmsa.Molecule(pdb=conf,top=topol,ndx=index,fms=[dense_dists])


In [ ]:
X_train = mol.intcoord(geom).T
X_train.shape